<a href="https://colab.research.google.com/github/nilesh3030/Stroke-Prediction/blob/main/Notebooks/1_Data_Cleaning_Steps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Stroke_Prediction/Data/stroke_case.csv', low_memory = False) ## Change the file name
pd.set_option ('display.max_columns', None)
pd.set_option ('display.max_rows', None)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
print(df.shape)
#df.head()
len(df.jaenan_sn.unique())

In [ ]:
# There are many junk columns that have been added so we will remove all of them 
df = df.loc[:,:'pti2']
print(df.shape)

In [ ]:
# Dedup the data 
df = df.drop_duplicates(subset=['jaenan_sn'])
print(df.shape)

In [ ]:
missing_variables = round((df.isna().sum()/len(df))*100,2)
missing_variables = missing_variables.loc[lambda x : x >= 30]

var_to_remove = missing_variables.index

len(var_to_remove)

In [ ]:
var_to_remove = [e for e in var_to_remove if e not in ('sx2', 'sx3', 'sx4', 'sx5')]

len(var_to_remove)

In [ ]:
data = df.drop(columns= var_to_remove)

In [ ]:
print(data.shape)

In [ ]:
# make list of numerical variables
num_vars = [var for var in data.columns if data[var].dtypes != 'O']

print('Number of numerical variables: ', len(num_vars))

# visualise the numerical variables
data[num_vars].head()

In [ ]:
# capture categorical variables in a list
cat_vars = [var for var in data.columns if data[var].dtypes == 'O']

print('Number of categorical variables: ', len(cat_vars))

# let's visualise the values of the categorical variables
data[cat_vars].head()

In [ ]:
cat_vars_final = ['cv_cc',
#'cv_etc',
'cv2_cc',
'cv2_phx_yn',
'cv2_act',
#'cv2_phss',
#'cv2_laphss',
#'cv2_laphss_asym',
#'cv2_laphss_seizure',
#'cv2_laphss_bst',
'sex',
'sx1',
'sx2',
'sx3',
'sx4',
'sx5',
'medical_history',
'stroke',
'obstacle2',
'cv2_cphss',
# Below are the disease variables that we are adding the for phase 2
'hypertension',
'diabetes',
'chest_disease',
'heart_disease',
'tuberculosis',
'hepatitis',
'liver',
'allergy',
'cancer',
'renal_failure',
'triage']

Google API will not work on the bulk data so we need to limit the data we pass in one go and hence we will first take the neccessary columns that we need and then make a list of unique values from these columns so that we can control the number of records that we pass through the Google API.

In [ ]:
unique_list = []
for column in cat_vars_final:
    unique = df[column].unique()
    for element in unique:
      unique_list.append(element)

In [ ]:
print(len(unique_list))
unique_list = set(unique_list)
print(len(unique_list))

## Below code block needs to be run only once to translate the data and then we will store transalation and use it directly

In [ ]:
'''
!pip install googletrans==4.0.0-rc1

from googletrans import Translator
import time
time.sleep(10)

translator = Translator()
translations = {}
for element in unique_list:
  #time.sleep(1)
  translations[element] = translator.translate(element).text

#for i in translations.items():
    #print(i) 



### Nan was transalated incorrectly so we will remove the Nan translation from dictionary as we would be trating Nan later
import numpy as np
print(len(translations))
translations.pop(np.nan)
print(len(translations))


### Updating the incorrect translations in the translation dictionary

translations.update({'심,뇌혈관계':'Heart, cerebrovascular',
'발음이상':'strange pronunciation',
'사지 저림':'numb feet and arms',
'음성':'negative',
'양성':'positive',
'남':'male',
'어지러움':'Dizziness',
'전신쇠약':'body weekness',
'오심':'misdiagnosis',
'심계항진':'Palpitations',
'질출혈':'Vaginal blooding'})


### Saving the translations file as json

import json

with open('/content/drive/MyDrive/Stroke_Prediction/Data/translations_2022.txt','w') as fp:
    fp.write(json.dumps(translations))
'''

In [ ]:
### Open the saved file to use
import json
with open('/content/drive/MyDrive/Stroke_Prediction/Data/translations_2022.txt','r') as json_file:
   translations = json.load(json_file)

In [ ]:
cat_vars_final.append('onset')
cat_vars_final.append('test1')
### Time and data variables
cat_vars_final.append('call_d')
cat_vars_final.append('call_t')
cat_vars_final.append('jaenan_sn')

In [ ]:
# final numerical variables based on our selection
num_vars_final = ['age',
'dbp1',
'sbp1',
'pr1',
'rr1',
'bt1',
'spo2_1',
'dbp2',
'sbp2',
'pr2',
'rr2',
'bt2',
'spo2_2']

In [ ]:
final_variables = []
for var in cat_vars_final:
  final_variables.append(var)

for var in num_vars_final:
  final_variables.append(var)

In [ ]:
# null values % in the final categorical and numerical columns
final_data = data[final_variables]
final_data.shape

In [ ]:
final_data.replace(translations, inplace=True)
print(final_data.cv2_cphss.unique())
final_data.cv2_cphss.value_counts()

In [ ]:
final_data.to_csv('/content/drive/MyDrive/Stroke_Prediction/Data/Stroke_clean_2022.csv', index = False)